## Econ 1923 - CPI project

In [1]:
import requests
import json
import pandas as pd

In [2]:
%run APIkeys.py
key = os.environ['BLS_API_key']

API keys loaded


### 2.1 Items and Weights
Your basket must include at least 8 ingredients.
https://download.bls.gov/pub/time.series/cu/cu.item

Student expenses
- SEEB01	College tuition and fees; weight: 0.4
- SERF01	Club membership for shopping clubs, fraternal, or other organizations, or participant sports fees; weight: 0.01
- SSEA011	College textbooks; weight: 0.02
- SEFP01	Coffee; weight: 0.01
- SAA	    Apparel; weight: 0.1
- SS62032	Admission to sports; weight: 0.06
- SEFV	    Food away from home; weight: 0.15
- SEFT02	Frozen food; weight: 0.25


- SAR     Recreation; 0.4
- SAS4    Transportation services; 0.25
- SAA     Apparel; 0.02
- SAM     Medical care; 0.01
- SEFV    food away from home; 0.15
- SAF11   food at home; 0.1
- SAF114  beverages; 0.01
- SERC02  sports equipment; 0.06



### 2.2 Locations and weights
The locations where these prices are measured. List at least three locations (cities/areas) based on where the team members either live now, lived in the past, or want to live in the future.2 Choose weights for these locations according to your liking.

- S12B    Philadelphia, PA; weight: 0.5
- S23A	  Chicago, IL; weight: 0.3
- S49D    Seattle, WA; weight: 0.2

### 2.3 Finalizing the basket
Combining the choices of items and locations, find all the series names. assign a weight for each series that combines the weights that you chose for the items and the weights you chose for the locations.

- College tuition and fees in Pittsburgh: CUURA104SEEB01
- Club membership Pittsburgh: CUURA104SERF01
- college textbooks Boston: CUURS11ASSEA011
- coffee boston: CUURS11ASEFP01
- apparel boston: CUURS11ASAA
- admission seattle: CUURS49DSS62031
- food away seattle: CUURS49DSEFV
- food at home seattle: CUURS49DSAF11



Finalized basket:
- CUURS23ASAR;       Recreation in Chicago; weight **0.23**
- CUUR0200SAS4;      Public transportation in Midwest; weight **0.18**
- CUURS12BSAA;       Apparel in the Northeast; weight **0.17**
- CUUR0100SAM;       Medicine NE; weight **0.05**
- CUURS49DSEFV       Food away in NE; weight **0.11**
- CUURS49DSAF111;      Food home in Seattle; weight **0.10**
- CUURS49DSAF114;    Non-alcoholic beverages in Seattle; weight **0.07**
- CUUR0000SERC02;    Sports equipment US; weight **0.09**


## 3. Downloading the data 
### 3.1 Refine the basket
First, verify that the series you created in the previous section exist in the BLS’ API. If some items do not exist, take them out and re-weight. If a certain location produces all the missing series, replace that location with a bigger or more general location.3
After you made all the changes and adjustments, state the final basket and the weights for each item in the basket. Table format will be great here.

In [3]:
def Series(varList,myKey):
    base_url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'  #this will not change
    headers = {'Content-type': 'application/json'}  #This will not changed !

    parameters = {
        "seriesid":varList,
        "startyear":"2012", 
        "endyear":"2022",
        "catalog":True, 
        "calculations":False, 
        "annualaverage":False,
        "aspects":False,
        "registrationkey": myKey 
     }

    data = json.dumps(parameters) #this converts the Python dictionary into a JSON format
    
    p = requests.post(base_url, data=data, headers=headers)
    json_data = json.loads(p.text)
    
    return json_data

In [4]:
# what items exist for philly: beverages, food away, food at home, apparel
basket2 = ['CUURS12BSAF114', 'CUURS12BSERF01','CUURS12BSSEA011', 'CUURS12BSEFP01', 'CUURS12BSAA', 
          'CUURS12BSS62032', 'CUURS12BSEFV', 'CUURS12BSAF11']

In [5]:
# what items exist for North East: beverages, food away, food at home, apparel
basket3  = ['CUUR0100SAF114', 'CUUR0100SERF01','CUUR0100SSEA011', 'CUUR0100SEFP01', 'CUUR0100SAA', 
          'CUUR0100SS62032', 'CUUR0100SEFV', 'CUUR0100SAF11']

In [6]:
# what items exist for Chicago: beverages, food away, food at home, apparel
basket4 = ['CUURS23ASAF114', 'CUURS23ASERF01','CUURS23ASSEA011', 'CUURS23ASEFP01', 'CUURS23ASAA', 
          'CUURS23ASS62032', 'CUURS23ASEFV', 'CUURS23ASAF11']

In [7]:
# what other items exist for Chicago: recreation
basket5 = ['CUURS23ASAR', 'CUURS23ASAS4','CUURS23ASEEE01', 'CUURS23ASEGC01', 'CUURS23ASEGD03', 
          'CUURS23ASERC02', 'CUURS23ASERG02', 'CUURS23ASETG', 'CUURS23ASSEE041', 'CUURS23ASS62032']

In [8]:
# what other items exist for midwest: recreation, transportation services
basket6 = ['CUUR0200SAR', 'CUUR0200SAS4','CUUR0200SEEE01', 'CUUR0200SEGC01', 'CUUR0200SEGD03', 
          'CUUR0200SERC02', 'CUUR0200SERG02', 'CUUR0200SETG', 'CUUR0200SSEE041', 'CUUR0200SS62032']

In [9]:
# what items exist for Seattle: beverages, food away, food at home, apparel
basket7  = ['CUURS49DSAF114', 'CUURS49DSERF01','CUURS49DSSEA011', 'CUURS49DSEFP01', 'CUURS49DSAA', 
          'CUURS49DSS62032', 'CUURS49DSEFV', 'CUURS49DSAF11']

In [10]:
# this chunk contains finalized basket
# recreation chicago, transportation services MW, apparel NE, medical care NE, food away seattle,
# food at home seattle, beverages seattle, sports equipment, total CPI
basket = ['CUURS23ASAR', 'CUUR0200SAS4', 'CUURS12BSAA', 'CUUR0100SAM', 'CUURS49DSEFV', 'CUURS49DSAF11', 
          'CUURS49DSAF114', 'CUUR0000SERC02', 'CUUR0000SA0']

In [11]:
result = Series(basket,key)

In [12]:
result['Results']['series']

[{'seriesID': 'CUURS23ASAR',
  'catalog': {'series_title': 'Recreation in Chicago-Naperville-Elgin, IL-IN-WI, all urban consumers, not seasonally adjusted',
   'series_id': 'CUURS23ASAR',
   'seasonality': 'Not Seasonally Adjusted',
   'survey_name': 'CPI for All Urban Consumers (CPI-U)',
   'survey_abbreviation': 'CU',
   'measure_data_type': 'Recreation',
   'area': 'Chicago-Naperville-Elgin, IL-IN-WI',
   'item': 'Recreation'},
  'data': [{'year': '2022',
    'period': 'M08',
    'periodName': 'August',
    'latest': 'true',
    'value': '123.405',
    'footnotes': [{}]},
   {'year': '2022',
    'period': 'M07',
    'periodName': 'July',
    'value': '120.838',
    'footnotes': [{}]},
   {'year': '2022',
    'period': 'M06',
    'periodName': 'June',
    'value': '120.646',
    'footnotes': [{}]},
   {'year': '2022',
    'period': 'M05',
    'periodName': 'May',
    'value': '120.756',
    'footnotes': [{}]},
   {'year': '2022',
    'period': 'M04',
    'periodName': 'April',
    'v

In [13]:
def Series2(varList,myKey):
    base_url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'  #this will not change
    headers = {'Content-type': 'application/json'}  #This will not changed !

    parameters = {
        "seriesid":varList,
        "startyear":"2012", 
        "endyear":"2022",
        "catalog":True, 
        "calculations":False, 
        "annualaverage":False,
        "aspects":False,
        "registrationkey": myKey 
     }

    data = json.dumps(parameters) #this converts the Python dictionary into a JSON format
    
    p = requests.post(base_url, data=data, headers=headers)
    json_data = json.loads(p.text)
    
    n = len(varList) #number of series requested
    
    for i in range(n):
        if json_data['Results']['series'][i]['data'] == []:
            print('Series ',varList[i],' does not exist')
        else:
            print('Series ',varList[i],' exists')
    
    return json_data

In [14]:
result2 = Series2(basket,key)

Series  CUURS23ASAR  exists
Series  CUUR0200SAS4  exists
Series  CUURS12BSAA  exists
Series  CUUR0100SAM  exists
Series  CUURS49DSEFV  exists
Series  CUURS49DSAF11  exists
Series  CUURS49DSAF114  exists
Series  CUUR0000SERC02  exists
Series  CUUR0000SA0  exists


### 3.2 Downloading
Make an API call to download the data for the basket you created. I ask that you have data for the last 5 years: January 2017 to January 2022. Make sure your API call retrieves data for this range.
Include the general CPI (CUUR0000SA0) in your API call.
Question: Are all the series you downloaded in a monthly frequency? If not, how can you fix that (i.e. interpolate).

In [15]:
def Series3(varList,myKey):
    base_url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'  #this will not change
    headers = {'Content-type': 'application/json'}  #This will not changed !

    parameters = {
        "seriesid":varList,
        "startyear":"2017", 
        "endyear":"2022",
        "catalog":True, 
        "calculations":False, 
        "annualaverage":False,
        "aspects":False,
        "registrationkey": myKey 
     }

    data = json.dumps(parameters) #this converts the Python dictionary into a JSON format
    
    p = requests.post(base_url, data=data, headers=headers)
    json_data = json.loads(p.text)
    
    n = len(varList) #number of series requested
    
    for i in range(n):
        if json_data['Results']['series'][i]['data'] == []:
            print('Series ',varList[i],' does not exist')
        else:
            print('Series ',varList[i],' exists')
    
    return json_data

In [16]:
result3 = Series3(basket,key)

Series  CUURS23ASAR  exists
Series  CUUR0200SAS4  exists
Series  CUURS12BSAA  exists
Series  CUUR0100SAM  exists
Series  CUURS49DSEFV  exists
Series  CUURS49DSAF11  exists
Series  CUURS49DSAF114  exists
Series  CUUR0000SERC02  exists
Series  CUUR0000SA0  exists


In [24]:
def Series4(varList,myKey):
    base_url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'  #this will not change
    headers = {'Content-type': 'application/json'}  #This will not changed !

    parameters = {
        "seriesid":varList,
        "startyear":"2017", 
        "endyear":"2022",
        "catalog":True, 
        "calculations":False, 
        "annualaverage":False,
        "aspects":False,
        "registrationkey": myKey 
     }

    data = json.dumps(parameters) #this converts the Python dictionary into a JSON format
    
    p = requests.post(base_url, data=data, headers=headers)
    json_data = json.loads(p.text)
    
    n = len(varList) #number of series requested
    
    new_df = pd.DataFrame(columns=['year', 'period']) # create an empty dataframe with two columns year and period
    
    for item in range(n):
        if json_data['Results']['series'][item]['data'] == []:
            print('Series ',varList[item],' does not exist')
        else:
            print('Series ',varList[item],' exists')
            d = json_data['Results']['series'][item]['data']
            current_df = pd.DataFrame(data=d)
            current_df = current_df[["year","period","value"]]
            current_df = current_df.rename(columns = {"value":varList[item]})
            
            new_df = new_df.merge(current_df, on = ['year','period'],how='outer')
    
    return new_df

In [27]:
myBasket = ['CUURS23ASAR', 'CUUR0200SAS4', 'CUURS12BSAA', 'CUUR0100SAM', 'CUURS49DSEFV', 'CUURS49DSAF11', 
          'CUURS49DSAF114', 'CUUR0000SERC02', 'CUUR0000SA0']
myData = Series4(myBasket,key)

Series  CUURS23ASAR  exists
Series  CUUR0200SAS4  exists
Series  CUURS12BSAA  exists
Series  CUUR0100SAM  exists
Series  CUURS49DSEFV  exists
Series  CUURS49DSAF11  exists
Series  CUURS49DSAF114  exists
Series  CUUR0000SERC02  exists
Series  CUUR0000SA0  exists


In [28]:
myData

,year,period,CUURS23ASAR,CUUR0200SAS4,CUURS12BSAA,CUUR0100SAM,CUURS49DSEFV,CUURS49DSAF11,CUURS49DSAF114,CUUR0000SERC02,CUUR0000SA0
0,2022,M08,123.405,348.590,112.289,580.669,391.714,317.676,396.774,87.153,296.171
1,2022,M07,120.838,348.248,NaN,576.156,NaN,317.686,NaN,86.546,296.276
2,2022,M06,120.646,353.242,108.908,573.145,377.700,311.685,389.261,87.473,296.311
3,2022,M05,120.756,352.820,NaN,568.877,NaN,310.451,NaN,87.348,292.296
4,2022,M04,120.529,340.863,108.442,568.472,372.197,306.625,387.016,87.491,289.109
...,...,...,...,...,...,...,...,...,...,...,...
63,2017,M05,111.163,308.659,NaN,491.976,NaN,249.725,NaN,81.440,244.733
64,2017,M04,113.185,307.524,113.182,491.537,296.316,248.565,NaN,82.046,244.524
65,2017,M03,113.501,304.921,NaN,493.595,NaN,249.135,NaN,81.941,243.801
66,2017,M02,111.345,303.383,114.340,492.978,294.503,247.070,NaN,82.001,243.603


### 4. Aggregating
Compute your version of the CPI based on the weights in 2.3.

CPI = 0.23(CUURS23ASAR) + 0.18(CUUR0200SAS4) + 0.17(CUURS12BSAA) + 0.05(CUUR0100SAM) + 0.11(CUURS49DSEFV) + 0.10(CUURS49DSAF111) + 0.07(CUURS49DSAF114) + 0.09(CUUR0000SERC02)


### 5. Inflation
Compute the 12-month inflation for several lags based on your CPI and compare it with inflation based on the general CPI. Compare and discuss the results.
Bonus: Put both on one graph and make sure the two lines are clearly labeled as well as the axes.
